## Comodo Exmaple 
This examples, load a basic robot model (i.e. composed only of basic shapes), modifies the links of such a robot model by elongating the legs, define instances of the TSID (Task Based Inverse Dynamics) and Centroidal MPC  controller and simulate the behavior of the robot using drake.  

In [28]:
# Comodo import
# from comodo.mujocoSimulator.mujocoSimulator import MujocoSimulator
# from comodo.drakeSimulator.drakeSimulator import DrakeSimulator
# from comodo.robotModel.robotModel import RobotModel
# from comodo.robotModel.createUrdf import createUrdf
# from comodo.centroidalMPC.centroidalMPC import CentroidalMPC
# from comodo.centroidalMPC.mpcParameterTuning import MPCParameterTuning
# from comodo.TSIDController.TSIDParameterTuning import TSIDParameterTuning
# from comodo.TSIDController.TSIDController import TSIDController

# ABSOLUTE IMPORTS FOR TESTING FAST
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('/home/laniakea/git/odio_urdf/')
from odio_urdf import *

sys.path.append('../src')
from comodo.mujocoSimulator.mujocoSimulator import MujocoSimulator
from comodo.drakeSimulator.drakeSimulator import DrakeSimulator
from comodo.robotModel.robotModel import RobotModel
from comodo.robotModel.createUrdf import createUrdf
from comodo.centroidalMPC.centroidalMPC import CentroidalMPC
from comodo.centroidalMPC.mpcParameterTuning import MPCParameterTuning
from comodo.TSIDController.TSIDParameterTuning import TSIDParameterTuning
from comodo.TSIDController.TSIDController import TSIDController

In [29]:
# General  import 
import xml.etree.ElementTree as ET
import numpy as np
import tempfile
import urllib.request
import logging

In [30]:
# imports for visualisation
# from pydrake.all import StartMeshcat
# meshcat = StartMeshcat()

In [31]:
# Getting stickbot urdf file and convert it to string 
urdf_robot_file = tempfile.NamedTemporaryFile(mode="w+")
url = 'https://raw.githubusercontent.com/icub-tech-iit/ergocub-gazebo-simulations/master/models/stickBot/model.urdf'
urllib.request.urlretrieve(url, urdf_robot_file.name)
# Load the URDF file
tree = ET.parse(urdf_robot_file.name)
root = tree.getroot()

# Convert the XML tree to a string
robot_urdf_string_original = ET.tostring(root)

create_urdf_instance = createUrdf(
    original_urdf_path=urdf_robot_file.name, save_gazebo_plugin=False
)

In [32]:
# Define parametric links and controlled joints  
legs_link_names = ["hip_3", "lower_leg"]
joint_name_list = [
    "r_shoulder_pitch",
    "r_shoulder_roll",
    "r_shoulder_yaw",
    "r_elbow",
    "l_shoulder_pitch",
    "l_shoulder_roll",
    "l_shoulder_yaw",
    "l_elbow",
    "r_hip_pitch",
    "r_hip_roll",
    "r_hip_yaw",
    "r_knee",
    "r_ankle_pitch",
    "r_ankle_roll",
    "l_hip_pitch",
    "l_hip_roll",
    "l_hip_yaw",
    "l_knee",
    "l_ankle_pitch",
    "l_ankle_roll",
]

In [33]:
# Define the robot modifications
modifications = {}
for item in legs_link_names:
    left_leg_item = "l_" + item
    right_leg_item = "r_" + item
    modifications.update({left_leg_item: 1.2})
    modifications.update({right_leg_item: 1.2})
# Motors Parameters 
Im_arms = 1e-3*np.ones(4) # from 0-4
Im_legs = 1e-3*np.ones(6) # from 5-10
kv_arms = 0.001*np.ones(4) # from 11-14
kv_legs = 0.001*np.ones(6) #from 20

Im = np.concatenate((Im_arms, Im_arms, Im_legs, Im_legs))
kv = np.concatenate((kv_arms, kv_arms, kv_legs, kv_legs))

In [34]:
# Modify the robot model and initialize
create_urdf_instance.modify_lengths(modifications)
urdf_robot_string = create_urdf_instance.write_urdf_to_file()
create_urdf_instance.reset_modifications()
robot_model_init = RobotModel(urdf_robot_string, "stickBot", joint_name_list)
s_des, xyz_rpy, H_b = robot_model_init.compute_desired_position_walking()

Unknown tag "sensor" in /robot[@name='stickBot']
Unknown tag "sensor" in /robot[@name='stickBot']
Unknown tag "sensor" in /robot[@name='stickBot']
Unknown tag "sensor" in /robot[@name='stickBot']
Unknown tag "sensor" in /robot[@name='stickBot']
Unknown tag "sensor" in /robot[@name='stickBot']
Unknown tag "sensor" in /robot[@name='stickBot']
Unknown tag "sensor" in /robot[@name='stickBot']
Unknown tag "sensor" in /robot[@name='stickBot']
Unknown tag "sensor" in /robot[@name='stickBot']


In [35]:
# Define simulator and initial position
drake_instance = DrakeSimulator()
drake_instance.set_visualize_robot_flag(True)
drake_instance.load_model(robot_model_init, s=s_des, xyz_rpy=xyz_rpy, kv_motors=kv, Im=Im)
s, ds, tau = drake_instance.get_state()
t = drake_instance.get_simulation_time()
H_b = drake_instance.get_base()
# beware of the velocity ordering
w_b = drake_instance.get_base_velocity()

INFO:drake:Meshcat listening for connections at http://localhost:7001


In [ ]:
# Define the controller parameters  and instantiate the controller
# Controller Parameters
tsid_parameter = TSIDParameterTuning()
mpc_parameters = MPCParameterTuning()

# TSID Instance 
TSID_controller_instance = TSIDController(frequency=0.01, robot_model=robot_model_init)
TSID_controller_instance.define_tasks(tsid_parameter)
TSID_controller_instance.set_state_with_base(s, ds, H_b, w_b, t)

# MPC Instance
step_lenght = 0.1
mpc = CentroidalMPC(robot_model=robot_model_init, step_length=step_lenght)
mpc.intialize_mpc(mpc_parameters=mpc_parameters)

# Set desired quantities
mpc.configure(s_init=s_des, H_b_init=H_b)
TSID_controller_instance.compute_com_position()
mpc.define_test_com_traj(TSID_controller_instance.COM.toNumPy())

# TODO: Set initial robot state and plan trajectories 
# mujoco_instance.step(1)

# Reading the state 
s, ds, tau = drake_instance.get_state()
H_b = drake_instance.get_base()
w_b = drake_instance.get_base_velocity()
t = drake_instance.get_simulation_time()

# MPC
mpc.set_state_with_base(s=s, s_dot=ds, H_b=H_b, w_b=w_b, t=t)
mpc.initialize_centroidal_integrator(s=s, s_dot=ds, H_b=H_b, w_b=w_b, t=t)
mpc_output = mpc.plan_trajectory()

In [ ]:
# set the logging level
logging.getLogger().setLevel(logging.INFO)

In [ ]:
# %autoreload 2

# from comodo.drakeSimulator.utils import DrakeURDFHelper
# urdf_string = robot_model_init.urdf_string
# duh = DrakeURDFHelper()
# duh.load_urdf(urdf_string=urdf_string)
# duh.remove_all_collisions()
# duh.fix_not_in_joint_list(robot_model_init.joint_name_list)
# duh.convert_xml_to_odio()
# duh.add_acutation_tags()
# # duh.odio_robot

# from pydrake.all import *
# builder = DiagramBuilder()
# time_step = 1e-3
# plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=time_step)
# parser = Parser(plant, scene_graph)
# robot_model_sim = parser.AddModels(
#             file_contents=str(duh.odio_robot),
#             file_type="urdf",
#         )[0]
# plant.Finalize()
# plant.set_name("plant")

# builder.ExportInput(plant.get_actuation_input_port(), "control_input")
# builder.ExportOutput(plant.get_state_output_port(), "state_output")

# diagram = builder.Build()
# simulator = Simulator(diagram)

# system = simulator.get_system()
# state_output_port = system.GetOutputPort("state_output")
# control_input_port = system.GetInputPort("control_input")

# context = simulator.get_mutable_context()
# context.SetTime(0)
# simulator.Initialize()

# simulator.get_system().SetDefaultContext(context)

# diagram = simulator.get_system()
# plant = diagram.GetSubsystemByName("plant")
# plant_context = diagram.GetMutableSubsystemContext(plant, context)
